In [ ]:
# 05_sp500_cs_tree_robustness.ipynb
# Robustness of cross-sectional SP500 tree vs momentum across multiple test windows

import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from xgboost import XGBRegressor

import joblib
from pathlib import Path


import optuna
from sklearn.ensemble import HistGradientBoostingRegressor

plt.style.use("seaborn-v0_8-darkgrid")

def sharpe_ratio_np(returns, freq: int = 252) -> float:
    """Simple annualized Sharpe on a 1D array of daily returns."""
    r = np.asarray(returns, dtype=float)
    if r.size == 0 or np.isclose(r.std(), 0.0):
        return 0.0
    return np.sqrt(freq) * r.mean() / r.std()


In [ ]:
PROJECT_ROOT = os.path.abspath("..")
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("PROJECT_ROOT:", PROJECT_ROOT)

%load_ext autoreload
%autoreload 2

from src.data_loading_cross import load_sp500_adj_close
from src.signals_cross import (
    make_cross_sectional_signals,
    build_cross_sectional_matrix,
    CROSS_FEATURES,
)

from src.backtest import (
    equity_curve_from_returns,
    cagr,
    annualized_vol,
    sharpe_ratio,
    max_drawdown,
)


In [ ]:
# --- Global transaction cost setting ---
# round-trip cost as a fraction of notional per 21-day "trade"
# 0.0005 = 5 bps, 0.001 = 10 bps, etc.
COST_BPS = 0.001  # tweak this to whatever you want to assume


In [ ]:
# Load SP500 panel as long as we can reasonably go
# (set force_download=True the first time if needed)
prices = load_sp500_adj_close(start="2000-01-01")

prices.info()
print("Price panel shape:", prices.shape)
print("Date range:", prices.index.min(), "->", prices.index.max())
print("Number of tickers:", len(prices.columns))

lookahead = 21  # ~1 month forward return

signals_df = make_cross_sectional_signals(prices, lookahead=lookahead)

print("Signals shape:", signals_df.shape)
print("Columns:", signals_df.columns.tolist())

dates_all = signals_df.index.get_level_values("date")
tickers_all = signals_df.index.get_level_values("ticker")

print("Signals date range:", dates_all.min(), "->", dates_all.max())
print("Unique tickers in signals:", len(np.unique(tickers_all)))

# Build big (X, y, dates, tickers) matrix
X, y, dates, tickers = build_cross_sectional_matrix(signals_df)

print("X shape:", X.shape)
print("y shape:", y.shape)
print("Feature names:", CROSS_FEATURES)
print("Min/max date:", dates.min(), "->", dates.max())
print("Num unique tickers:", np.unique(tickers).size)

# Quick sanity check on forward-return distribution
plt.figure(figsize=(6, 4))
plt.hist(y, bins=100)
plt.title(f"Distribution of {lookahead}-day forward returns")
plt.xlabel("Forward return")
plt.ylabel("Frequency")
plt.show()


In [ ]:

symbols_path = Path(PROJECT_ROOT) / "data" / "sp500_symbols.csv"
symbols_df = pd.read_csv(symbols_path)

if "sector" in symbols_df.columns:
    sector_map = symbols_df.set_index("symbol")["sector"].to_dict()

    # signals_df has MultiIndex (date, ticker)
    signals_df["sector"] = (
        signals_df.index.get_level_values("ticker")
        .map(sector_map)
        .fillna("ALL")
    )
    print("Added sector column to signals_df.")
else:
    sector_map = None
    print(
        "Warning: sp500_symbols.csv has no 'sector' column – "
        "sector-neutral experiments will be skipped."
    )


In [ ]:
def daily_momentum_cs(
    group: pd.DataFrame,
    q: float = 0.1,
    horizon: int = 21,
    cost_bps: float = 0.0,
) -> pd.Series:
    """
    Cross-sectional momentum for a single date.

    group: rows for one date, many tickers.
    q: top/bottom quantile, e.g. 0.1 for deciles.
    horizon: forward horizon used for the target (e.g. 21 days).
    cost_bps: round-trip cost per 21-day position.
    """
    n = len(group)
    if n < 10:
        return pd.Series({"eqw": 0.0, "long": 0.0, "long_short": 0.0})

    # equal-weight all stocks -> "index" (21-day return, frictionless)
    eqw_ret_21 = group["target_fwd_21"].mean()

    # sort by past 21d return (classical cross-sectional momentum)
    g_sorted = group.sort_values("ret_21")
    k = max(1, int(n * q))

    bottom = g_sorted.iloc[:k]
    top    = g_sorted.iloc[-k:]

    long_ret_21  = top["target_fwd_21"].mean()
    short_ret_21 = bottom["target_fwd_21"].mean()
    long_short_21 = long_ret_21 - short_ret_21

    # Apply transaction costs on 21-day horizon
    if cost_bps > 0.0:
        long_ret_21   = (1.0 + long_ret_21) * (1.0 - cost_bps) - 1.0
        long_short_21 = (1.0 + long_short_21) * (1.0 - 2.0 * cost_bps) - 1.0

    # convert to daily
    def to_daily(R):
        return (1.0 + R) ** (1.0 / horizon) - 1.0

    eqw_ret_daily    = to_daily(eqw_ret_21)
    long_ret_daily   = to_daily(long_ret_21)
    long_short_daily = to_daily(long_short_21)

    return pd.Series(
        {
            "eqw": eqw_ret_daily,
            "long": long_ret_daily,
            "long_short": long_short_daily,
        }
    )


def compute_cs_daily_returns(
    df: pd.DataFrame,
    q: float = 0.1,
    horizon: int = 21,
    cost_bps: float = 0.0,
):
    """
    df: index (date, symbol), columns: y_true, y_pred (21d fwd returns + predictions)
    Returns three Series of *daily-equivalent* returns:
        eqw, long-only (top q), long-short (top q minus bottom q)

    cost_bps:
        round-trip transaction cost per 21-day holding period as fraction
        e.g. 0.0005 = 5 bps, 0.001 = 10 bps.
        Applied once for a long-only position and twice for a long-short position.
    """

    def _per_date(group: pd.DataFrame) -> pd.Series:
        n = len(group)
        if n < 10:
            return pd.Series({"eqw": 0.0, "long": 0.0, "long_short": 0.0})

        # Equal-weight all stocks -> benchmark (frictionless)
        eqw_ret_21 = group["y_true"].mean()

        # Sort by predicted forward return
        g_sorted = group.sort_values("y_pred")
        k = max(1, int(n * q))

        bottom = g_sorted.iloc[:k]     # worst predicted
        top    = g_sorted.iloc[-k:]    # best predicted

        long_ret_21  = top["y_true"].mean()
        short_ret_21 = bottom["y_true"].mean()

        long_short_21 = long_ret_21 - short_ret_21

        # apply transaction costs
        if cost_bps > 0.0:
            long_ret_21   = (1.0 + long_ret_21) * (1.0 - cost_bps) - 1.0
            long_short_21 = (1.0 + long_short_21) * (1.0 - 2.0 * cost_bps) - 1.0

        def to_daily(R):
            return (1.0 + R) ** (1.0 / horizon) - 1.0

        eqw_daily        = to_daily(eqw_ret_21)
        long_daily       = to_daily(long_ret_21)
        long_short_daily = to_daily(long_short_21)

        return pd.Series(
            {"eqw": eqw_daily, "long": long_daily, "long_short": long_short_daily}
        )

    daily = df.groupby("date").apply(_per_date)

    eqw = daily["eqw"].astype(float)
    long = daily["long"].astype(float)
    long_short = daily["long_short"].astype(float)

    return eqw, long, long_short


In [ ]:
def compute_cs_daily_returns_gated(
    df: pd.DataFrame,
    q: float = 0.1,
    horizon: int = 21,
    cost_bps: float = 0.0,
):
    """
    Like compute_cs_daily_returns, but:
    - uses model predictions to decide whether to trade at all,
    - only applies costs on days where we actually trade.

    df: index (date, symbol), columns: y_true, y_pred
    """

    def _per_date(group: pd.DataFrame) -> pd.Series:
        n = len(group)
        if n < 10:
            return pd.Series({"eqw": 0.0, "long": 0.0, "long_short": 0.0})

        # Benchmark: equal-weight all stocks (no costs)
        eqw_ret_21 = group["y_true"].mean()

        # Rank by predicted forward return
        g_sorted = group.sort_values("y_pred")
        k = max(1, int(n * q))

        bottom = g_sorted.iloc[:k]
        top    = g_sorted.iloc[-k:]

        # --- True 21d returns (what actually happens) ---
        long_true_21  = top["y_true"].mean()
        short_true_21 = bottom["y_true"].mean()
        long_short_true_21 = long_true_21 - short_true_21

        # --- Predicted 21d returns (model view) ---
        long_pred_21  = top["y_pred"].mean()
        short_pred_21 = bottom["y_pred"].mean()
        long_short_pred_21 = long_pred_21 - short_pred_21

        # ---- Decision: do we trade? ----
        # Long-only: require predicted edge > cost
        trade_long = long_pred_21 > cost_bps

        # Long-short: require predicted spread > 2 * cost
        trade_ls   = long_short_pred_21 > 2.0 * cost_bps

        # Start from "no trade"
        long_ret_21      = 0.0
        long_short_21    = 0.0

        if trade_long:
            long_ret_21 = long_true_21

        if trade_ls:
            long_short_21 = long_short_true_21

        # ---- Apply costs only if we actually trade ----
        if cost_bps > 0.0 and trade_long:
            long_ret_21 = (1.0 + long_ret_21) * (1.0 - cost_bps) - 1.0

        if cost_bps > 0.0 and trade_ls:
            long_short_21 = (1.0 + long_short_21) * (1.0 - 2.0 * cost_bps) - 1.0

        # Convert 21-day returns to daily-equivalent
        def to_daily(R):
            return (1.0 + R) ** (1.0 / horizon) - 1.0

        eqw_daily        = to_daily(eqw_ret_21)
        long_daily       = to_daily(long_ret_21)
        long_short_daily = to_daily(long_short_21)

        return pd.Series(
            {"eqw": eqw_daily, "long": long_daily, "long_short": long_short_daily}
        )

    daily = df.groupby("date").apply(_per_date)

    eqw = daily["eqw"].astype(float)
    long = daily["long"].astype(float)
    long_short = daily["long_short"].astype(float)

    return eqw, long, long_short


In [ ]:
def daily_momentum_cs_sector_neutral(
    group: pd.DataFrame,
    q: float = 0.1,
    horizon: int = 21,
    cost_bps: float = 0.0,
) -> pd.Series:
    """
    Sector-neutral cross-sectional momentum for a single date.

    group: rows for one date, many tickers, with columns:
        target_fwd_21, ret_21, sector
    """

    n = len(group)
    if n < 10 or "sector" not in group.columns:
        # Fall back to non-sector version
        return daily_momentum_cs(group, q=q, horizon=horizon, cost_bps=cost_bps)

    # Benchmark (frictionless)
    eqw_ret_21 = group["target_fwd_21"].mean()

    sector_long = []
    sector_short = []

    for sec, g_sec in group.groupby("sector"):
        m = len(g_sec)
        if m < 5:
            continue

        g_sorted = g_sec.sort_values("ret_21")
        k = max(1, int(m * q))

        bottom = g_sorted.iloc[:k]
        top    = g_sorted.iloc[-k:]

        sector_long.append(top["target_fwd_21"].mean())
        sector_short.append(bottom["target_fwd_21"].mean())

    if not sector_long:
        return pd.Series({"eqw": 0.0, "long": 0.0, "long_short": 0.0})

    long_ret_21  = float(np.mean(sector_long))
    short_ret_21 = float(np.mean(sector_short))
    long_short_21 = long_ret_21 - short_ret_21

    if cost_bps > 0.0:
        long_ret_21   = (1.0 + long_ret_21) * (1.0 - cost_bps) - 1.0
        long_short_21 = (1.0 + long_short_21) * (1.0 - 2.0 * cost_bps) - 1.0

    def to_daily(R):
        return (1.0 + R) ** (1.0 / horizon) - 1.0

    eqw_daily    = to_daily(eqw_ret_21)
    long_daily   = to_daily(long_ret_21)
    ls_daily     = to_daily(long_short_21)

    return pd.Series({"eqw": eqw_daily, "long": long_daily, "long_short": ls_daily})


def compute_cs_daily_returns_sector_neutral(
    df: pd.DataFrame,
    q: float = 0.1,
    horizon: int = 21,
    cost_bps: float = 0.0,
):
    """
    Sector-neutral version of compute_cs_daily_returns.
    df: index (date, symbol), columns: y_true, y_pred, sector.
    """

    if "sector" not in df.columns:
        # Graceful fallback
        return compute_cs_daily_returns(df, q=q, horizon=horizon, cost_bps=cost_bps)

    def _per_date(group: pd.DataFrame) -> pd.Series:
        n = len(group)
        if n < 10:
            return pd.Series({"eqw": 0.0, "long": 0.0, "long_short": 0.0})

        eqw_ret_21 = group["y_true"].mean()

        sector_long = []
        sector_short = []

        for sec, g_sec in group.groupby("sector"):
            m = len(g_sec)
            if m < 5:
                continue

            g_sorted = g_sec.sort_values("y_pred")
            k = max(1, int(m * q))

            bottom = g_sorted.iloc[:k]
            top    = g_sorted.iloc[-k:]

            sector_long.append(top["y_true"].mean())
            sector_short.append(bottom["y_true"].mean())

        if not sector_long:
            return pd.Series({"eqw": 0.0, "long": 0.0, "long_short": 0.0})

        long_ret_21  = float(np.mean(sector_long))
        short_ret_21 = float(np.mean(sector_short))
        long_short_21 = long_ret_21 - short_ret_21

        if cost_bps > 0.0:
            long_ret_21   = (1.0 + long_ret_21) * (1.0 - cost_bps) - 1.0
            long_short_21 = (1.0 + long_short_21) * (1.0 - 2.0 * cost_bps) - 1.0

        def to_daily(R):
            return (1.0 + R) ** (1.0 / horizon) - 1.0

        eqw_daily    = to_daily(eqw_ret_21)
        long_daily   = to_daily(long_ret_21)
        ls_daily     = to_daily(long_short_21)

        return pd.Series({"eqw": eqw_daily, "long": long_daily, "long_short": ls_daily})

    daily = df.groupby("date").apply(_per_date)
    eqw = daily["eqw"].astype(float)
    long = daily["long"].astype(float)
    long_short = daily["long_short"].astype(float)
    return eqw, long, long_short


In [ ]:
def run_cs_window(
    test_start: str,
    test_end: str,
    X: np.ndarray,
    y: np.ndarray,
    dates: pd.Series,
    tickers: np.ndarray,
    signals_df: pd.DataFrame,
    n_trials: int = 10,
    q_mom: float = 0.1,
    cost_bps: float = COST_BPS,
    random_state: int = 42,
):
    """
    Run the full pipeline for a single test window:
    - Train/val on data *before* test_start
    - Tune tree + XGB + q on validation long-short Sharpe (net of costs)
    - Evaluate momentum, tree, and XGB on [test_start, test_end]
    """
    test_start = pd.Timestamp(test_start)
    test_end   = pd.Timestamp(test_end)

    # Masks for this window
    test_mask = (dates >= test_start) & (dates <= test_end)
    hist_mask = dates < test_start

    if test_mask.sum() == 0 or hist_mask.sum() < 1000:
        print(f"Skipping window {test_start.date()}–{test_end.date()} (not enough data).")
        return None

    # 70/30 split of *history* into train/val
    hist_dates = np.array(sorted(dates[hist_mask].unique()))
    train_end_local = hist_dates[int(len(hist_dates) * 0.7)]

    train_mask = (dates <= train_end_local)
    val_mask   = (dates > train_end_local) & (dates < test_start)

    X_train, y_train = X[train_mask], y[train_mask]
    X_val,   y_val   = X[val_mask],   y[val_mask]
    X_test,  y_test  = X[test_mask],  y[test_mask]

    dates_val_local    = dates[val_mask]
    dates_test_local   = dates[test_mask]
    tickers_val_local  = tickers[val_mask]
    tickers_test_local = tickers[test_mask]

    print(
        f"Window {test_start.date()}–{test_end.date()} | "
        f"train={len(y_train)}, val={len(y_val)}, test={len(y_test)}"
    )

    # -------------------------
    # Optuna objectives
    # -------------------------
    def objective_tree_cs(trial):
        max_depth        = trial.suggest_int("max_depth", 2, 8)
        learning_rate    = trial.suggest_float("learning_rate", 0.01, 0.2, log=True)
        max_iter         = trial.suggest_int("max_iter", 100, 500)
        min_samples_leaf = trial.suggest_int("min_samples_leaf", 20, 200)
        q                = trial.suggest_float("q", 0.05, 0.3)  # top/bottom 5–30%

        model = HistGradientBoostingRegressor(
            max_depth=max_depth,
            learning_rate=learning_rate,
            max_iter=max_iter,
            min_samples_leaf=min_samples_leaf,
            random_state=random_state,
        )
        model.fit(X_train, y_train)

        y_pred_val = model.predict(X_val)

        df_val = (
            pd.DataFrame(
                {
                    "date":   dates_val_local,
                    "symbol": tickers_val_local,
                    "y_true": y_val,
                    "y_pred": y_pred_val,
                }
            )
            .set_index(["date", "symbol"])
            .sort_index()
        )

        # long-short (NET of costs)
        _, _, long_short_val = compute_cs_daily_returns(
            df_val,
            q=q,
            horizon=lookahead,
            cost_bps=cost_bps,
        )

        ret_series = long_short_val.replace([np.inf, -np.inf], np.nan).dropna()
        if len(ret_series) < 20:
            return 0.0

        return -sharpe_ratio_np(ret_series.values)

    def objective_xgb_cs(trial):
        # Reasonable XGB hyperparameter ranges
        max_depth        = trial.suggest_int("max_depth", 2, 8)
        learning_rate    = trial.suggest_float("learning_rate", 0.01, 0.3, log=True)
        n_estimators     = trial.suggest_int("n_estimators", 100, 600)
        subsample        = trial.suggest_float("subsample", 0.6, 1.0)
        colsample_bytree = trial.suggest_float("colsample_bytree", 0.6, 1.0)
        min_child_weight = trial.suggest_float("min_child_weight", 1.0, 20.0)
        q                = trial.suggest_float("q", 0.05, 0.3)

        model = XGBRegressor(
            max_depth=max_depth,
            learning_rate=learning_rate,
            n_estimators=n_estimators,
            subsample=subsample,
            colsample_bytree=colsample_bytree,
            min_child_weight=min_child_weight,
            objective="reg:squarederror",
            tree_method="hist",
            n_jobs=-1,
            random_state=random_state,
        )
        model.fit(X_train, y_train)

        y_pred_val = model.predict(X_val)
        df_val = (
            pd.DataFrame(
                {
                    "date":   dates_val_local,
                    "symbol": tickers_val_local,
                    "y_true": y_val,
                    "y_pred": y_pred_val,
                }
            )
            .set_index(["date", "symbol"])
            .sort_index()
        )

        _, _, long_short_val = compute_cs_daily_returns(
            df_val,
            q=q,
            horizon=lookahead,
            cost_bps=cost_bps,
        )

        ret_series = long_short_val.replace([np.inf, -np.inf], np.nan).dropna()
        if len(ret_series) < 20:
            return 0.0

        return -sharpe_ratio_np(ret_series.values)

    # --- tune tree ---
    study = optuna.create_study(direction="minimize")
    study.optimize(objective_tree_cs, n_trials=n_trials)
    print("  Tree best params:", study.best_params)
    print("  Tree best val -Sharpe:", study.best_value)

    best_params = study.best_params.copy()
    q_best = best_params.pop("q")

    # --- tune XGB ---
    study_xgb = optuna.create_study(direction="minimize")
    study_xgb.optimize(objective_xgb_cs, n_trials=n_trials)
    print("  XGB best params:", study_xgb.best_params)
    print("  XGB best val -Sharpe:", study_xgb.best_value)

    best_params_xgb = study_xgb.best_params.copy()
    q_best_xgb = best_params_xgb.pop("q")

    # Train on all history (train+val) before test_start
    hist_mask_full = dates < test_start
    X_hist, y_hist = X[hist_mask_full], y[hist_mask_full]

    tree_best = HistGradientBoostingRegressor(
        **best_params,
        random_state=random_state,
    )
    tree_best.fit(X_hist, y_hist)

    xgb_best = XGBRegressor(
        **best_params_xgb,
        objective="reg:squarederror",
        tree_method="hist",
        n_jobs=-1,
        random_state=random_state,
    )
    xgb_best.fit(X_hist, y_hist)

    # -------------------------
    # 1) Momentum baseline on this test window (NET)
    # -------------------------
    idx_dates = signals_df.index.get_level_values("date")
    signals_test = signals_df.loc[
        (idx_dates >= test_start) & (idx_dates <= test_end)
    ].copy()

    daily_mom = signals_test.groupby("date").apply(
        daily_momentum_cs,
        q=q_mom,
        horizon=lookahead,
        cost_bps=cost_bps,
    )

    eqw_returns   = daily_mom["eqw"].astype(float)
    momL_returns  = daily_mom["long"].astype(float)
    momLS_returns = daily_mom["long_short"].astype(float)

    # -------------------------
    # 2) Tree strategy on this test window (NET)
    # -------------------------
    y_pred_test = tree_best.predict(X_test)

    df_test = (
        pd.DataFrame(
            {
                "date":   dates_test_local,
                "symbol": tickers_test_local,
                "y_true": y_test,
                "y_pred": y_pred_test,
            }
        )
        .set_index(["date", "symbol"])
        .sort_index()
    )

    _, treeL_returns, treeLS_returns = compute_cs_daily_returns(
        df_test, q=q_best, horizon=lookahead, cost_bps=cost_bps
    )

    # -------------------------
    # 3) XGB strategy on this test window (NET)
    # -------------------------
    y_pred_test_xgb = xgb_best.predict(X_test)
    df_test_xgb = (
        pd.DataFrame(
            {
                "date":   dates_test_local,
                "symbol": tickers_test_local,
                "y_true": y_test,
                "y_pred": y_pred_test_xgb,
            }
        )
        .set_index(["date", "symbol"])
        .sort_index()
    )

    _, xgbL_returns, xgbLS_returns = compute_cs_daily_returns(
        df_test_xgb,
        q=q_best_xgb,
        horizon=lookahead,
        cost_bps=cost_bps,
    )

    # Equity curves
    eqw_eq     = equity_curve_from_returns(eqw_returns)
    momL_eq    = equity_curve_from_returns(momL_returns)
    momLS_eq   = equity_curve_from_returns(momLS_returns)
    treeL_eq   = equity_curve_from_returns(treeL_returns)
    treeLS_eq  = equity_curve_from_returns(treeLS_returns)
    xgbL_eq    = equity_curve_from_returns(xgbL_returns)
    xgbLS_eq   = equity_curve_from_returns(xgbLS_returns)

    # ---------- Sector-neutral versions (if sector_map is available) ----------
    if "sector_map" in globals() and sector_map is not None:
        df_test_tree_sn = df_test.copy()
        df_test_tree_sn["sector"] = (
            df_test_tree_sn.index.get_level_values("symbol")
            .map(sector_map)
            .fillna("ALL")
        )
        _, treeL_sn_returns, treeLS_sn_returns = compute_cs_daily_returns_sector_neutral(
            df_test_tree_sn,
            q=q_best,
            horizon=lookahead,
            cost_bps=cost_bps,
        )

        df_test_xgb_sn = df_test_xgb.copy()
        df_test_xgb_sn["sector"] = (
            df_test_xgb_sn.index.get_level_values("symbol")
            .map(sector_map)
            .fillna("ALL")
        )
        _, xgbL_sn_returns, xgbLS_sn_returns = compute_cs_daily_returns_sector_neutral(
            df_test_xgb_sn,
            q=q_best_xgb,
            horizon=lookahead,
            cost_bps=cost_bps,
        )
    else:
        # fallback: reuse non-sector-neutral returns
        treeL_sn_returns  = treeL_returns
        treeLS_sn_returns = treeLS_returns
        xgbL_sn_returns   = xgbL_returns
        xgbLS_sn_returns  = xgbLS_returns

    # -------------------------
    # Metrics dict for this window
    # -------------------------
    metrics = {
        "test_start": test_start.date(),
        "test_end":   test_end.date(),

        "momL_cagr":    cagr(momL_eq),
        "momL_sharpe":  sharpe_ratio(momL_returns),
        "momL_max_dd":  max_drawdown(momL_eq),

        "treeL_cagr":   cagr(treeL_eq),
        "treeL_sharpe": sharpe_ratio(treeL_returns),
        "treeL_max_dd": max_drawdown(treeL_eq),

        "momLS_cagr":   cagr(momLS_eq),
        "momLS_sharpe": sharpe_ratio(momLS_returns),
        "momLS_max_dd": max_drawdown(momLS_eq),

        "treeLS_cagr":   cagr(treeLS_eq),
        "treeLS_sharpe": sharpe_ratio(treeLS_returns),
        "treeLS_max_dd": max_drawdown(treeLS_eq),

        "xgbL_cagr":    cagr(xgbL_eq),
        "xgbL_sharpe":  sharpe_ratio(xgbL_returns),
        "xgbLS_cagr":   cagr(xgbLS_eq),
        "xgbLS_sharpe": sharpe_ratio(xgbLS_returns),

        "treeL_sn_sharpe":  sharpe_ratio(treeL_sn_returns),
        "treeLS_sn_sharpe": sharpe_ratio(treeLS_sn_returns),
        "xgbL_sn_sharpe":   sharpe_ratio(xgbL_sn_returns),
        "xgbLS_sn_sharpe":  sharpe_ratio(xgbLS_sn_returns),
    }

    # quick plot for this window
    plt.figure(figsize=(9, 4))
    momL_eq.plot(label="Momentum long-only (net)")
    treeL_eq.plot(label="Tree long-only (net)")
    xgbL_eq.plot(label="XGB long-only (net)", linestyle=":")
    plt.legend()
    plt.title(f"Long-only (net of costs): {test_start.date()}–{test_end.date()}")
    plt.show()

    plt.figure(figsize=(9, 4))
    momLS_eq.plot(label="Momentum long-short (net)")
    treeLS_eq.plot(label="Tree long-short (net)")
    xgbLS_eq.plot(label="XGB long-short (net)", linestyle=":")
    plt.legend()
    plt.title(f"Long-short (net of costs): {test_start.date()}–{test_end.date()}")
    plt.show()

    return metrics


In [ ]:
test_windows = [
    ("2005-01-01", "2009-12-31"),
    ("2010-01-01", "2014-12-31"),
    ("2015-01-01", "2019-12-31"),
    ("2020-01-01", "2024-12-31"),
]

results = []
for start, end in test_windows:
    print("=" * 80)
    m = run_cs_window(
        start,
        end,
        X=X,
        y=y,
        dates=dates,
        tickers=tickers,
        signals_df=signals_df,
        n_trials=10,
        q_mom=0.1,
    )
    if m is not None:
        results.append(m)




In [ ]:
# --------------------------------------------------------------------
# Aggregate results across windows and build summary tables / plots
# --------------------------------------------------------------------
results_df = pd.DataFrame(results)
results_df

summary_cols = [
    "test_start", "test_end",
    "momL_sharpe", "treeL_sharpe", "xgbL_sharpe",
    "momLS_sharpe", "treeLS_sharpe", "xgbLS_sharpe",
    "treeL_sn_sharpe", "xgbL_sn_sharpe",
    "treeLS_sn_sharpe", "xgbLS_sn_sharpe",
]

summary = results_df[summary_cols].copy()
summary

# Differences vs momentum
summary["treeL_minus_momL"]   = summary["treeL_sharpe"]  - summary["momL_sharpe"]
summary["treeLS_minus_momLS"] = summary["treeLS_sharpe"] - summary["momLS_sharpe"]
summary["xgbL_minus_momL"]    = summary["xgbL_sharpe"]   - summary["momL_sharpe"]
summary["xgbLS_minus_momLS"]  = summary["xgbLS_sharpe"]  - summary["momLS_sharpe"]

summary

# Nice labels for windows
window_labels = (
    results_df["test_start"].astype(str)
    + "–"
    + results_df["test_end"].astype(str)
)
x = np.arange(len(results_df))
width = 0.25  # 3 bars per window

# --- Long-only Sharpe: momentum vs tree vs XGB ---
plt.figure(figsize=(10, 4))
plt.bar(x - width,   results_df["momL_sharpe"],  width, label="Momentum long-only")
plt.bar(x,           results_df["treeL_sharpe"], width, label="Tree long-only")
plt.bar(x + width,   results_df["xgbL_sharpe"],  width, label="XGB long-only")
plt.xticks(x, window_labels, rotation=30, ha="right")
plt.ylabel("Sharpe")
plt.title("Long-only Sharpe by test window (net of costs)")
plt.legend()
plt.tight_layout()
plt.show()

# --- Long-short Sharpe: momentum vs tree vs XGB ---
plt.figure(figsize=(10, 4))
plt.bar(x - width,   results_df["momLS_sharpe"], width, label="Momentum long-short")
plt.bar(x,           results_df["treeLS_sharpe"], width, label="Tree long-short")
plt.bar(x + width,   results_df["xgbLS_sharpe"],  width, label="XGB long-short")
plt.xticks(x, window_labels, rotation=30, ha="right")
plt.ylabel("Sharpe")
plt.title("Long-short Sharpe by test window (net of costs)")
plt.legend()
plt.tight_layout()
plt.show()

# --- Δ Sharpe vs momentum (long-only) ---
plt.figure(figsize=(10, 4))
plt.bar(x - width/2, summary["treeL_minus_momL"], width, label="Tree - momentum")
plt.bar(x + width/2, summary["xgbL_minus_momL"],  width, label="XGB - momentum")
plt.axhline(0, color="black", linewidth=1)
plt.xticks(x, window_labels, rotation=30, ha="right")
plt.ylabel("Δ Sharpe")
plt.title("Long-only: ML vs momentum (per window)")
plt.legend()
plt.tight_layout()
plt.show()

# --- Δ Sharpe vs momentum (long-short) ---
plt.figure(figsize=(10, 4))
plt.bar(x - width/2, summary["treeLS_minus_momLS"], width, label="Tree - momentum")
plt.bar(x + width/2, summary["xgbLS_minus_momLS"],  width, label="XGB - momentum")
plt.axhline(0, color="black", linewidth=1)
plt.xticks(x, window_labels, rotation=30, ha="right")
plt.ylabel("Δ Sharpe")
plt.title("Long-short: ML vs momentum (per window)")
plt.legend()
plt.tight_layout()
plt.show()

# --- Long-only scatter: momentum vs tree, momentum vs XGB ---
plt.figure(figsize=(5, 5))
plt.scatter(results_df["momL_sharpe"], results_df["treeL_sharpe"], label="Tree")
plt.scatter(results_df["momL_sharpe"], results_df["xgbL_sharpe"],  label="XGB", marker="x")
lims = [
    min(results_df[["momL_sharpe", "treeL_sharpe", "xgbL_sharpe"]].min()) - 0.2,
    max(results_df[["momL_sharpe", "treeL_sharpe", "xgbL_sharpe"]].max()) + 0.2,
]
plt.plot(lims, lims, "--", color="gray")  # y = x line
for i, label in enumerate(window_labels):
    plt.text(
        results_df["momL_sharpe"].iloc[i] + 0.02,
        results_df["treeL_sharpe"].iloc[i] + 0.02,
        label,
        fontsize=8,
    )
plt.xlim(lims)
plt.ylim(lims)
plt.xlabel("Momentum long-only Sharpe")
plt.ylabel("ML long-only Sharpe")
plt.title("Long-only: momentum vs ML across windows")
plt.legend()
plt.tight_layout()
plt.show()

# --- Long-short scatter: momentum vs tree, momentum vs XGB ---
plt.figure(figsize=(5, 5))
plt.scatter(results_df["momLS_sharpe"], results_df["treeLS_sharpe"], label="Tree")
plt.scatter(results_df["momLS_sharpe"], results_df["xgbLS_sharpe"],  label="XGB", marker="x")
lims = [
    min(results_df[["momLS_sharpe", "treeLS_sharpe", "xgbLS_sharpe"]].min()) - 0.2,
    max(results_df[["momLS_sharpe", "treeLS_sharpe", "xgbLS_sharpe"]].max()) + 0.2,
]
plt.plot(lims, lims, "--", color="gray")
for i, label in enumerate(window_labels):
    plt.text(
        results_df["momLS_sharpe"].iloc[i] + 0.02,
        results_df["treeLS_sharpe"].iloc[i] + 0.02,
        label,
        fontsize=8,
    )
plt.xlim(lims)
plt.ylim(lims)
plt.xlabel("Momentum long-short Sharpe")
plt.ylabel("ML long-short Sharpe")
plt.title("Long-short: momentum vs ML across windows")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# === PAPER TRADING MODE, FOR RESEARCH PURPOSE ONLY: cross-sectional SP500 picks for "tomorrow" ===

from IPython.display import display

TOP_N_LONG = 20   # how many longs to show
TOP_N_SHORT = 20  # how many shorts to show

# 1) Choose as-of date: last date where we have signals/targets
as_of_date = dates.max()
print(f"As-of date for paper trading: {as_of_date.date()}")

# ----- build train / val sets using only history before as_of_date -----
hist_mask = dates < as_of_date
if hist_mask.sum() < 1000:
    raise ValueError("Not enough history before as_of_date for a meaningful model.")

hist_dates = np.array(sorted(dates[hist_mask].unique()))
train_end_live = hist_dates[int(len(hist_dates) * 0.7)]

train_mask_live = (dates <= train_end_live)
val_mask_live   = (dates > train_end_live) & (dates < as_of_date)

X_train_live, y_train_live = X[train_mask_live], y[train_mask_live]
X_val_live,   y_val_live   = X[val_mask_live],   y[val_mask_live]

dates_val_live   = dates[val_mask_live]
tickers_val_live = tickers[val_mask_live]

print(
    f"Live setup | train={len(y_train_live)} samples "
    f"val={len(y_val_live)} samples (all < {as_of_date.date()})"
)

# 2) Optuna objective: tune tree hyperparams + q on long-short Sharpe (net of cost)
def objective_tree_live(trial):
    max_depth       = trial.suggest_int("max_depth", 2, 8)
    learning_rate   = trial.suggest_float("learning_rate", 0.01, 0.2, log=True)
    max_iter        = trial.suggest_int("max_iter", 100, 500)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 20, 200)
    q               = trial.suggest_float("q", 0.05, 0.3)  # top/bottom 5–30%

    model = HistGradientBoostingRegressor(
        max_depth=max_depth,
        learning_rate=learning_rate,
        max_iter=max_iter,
        min_samples_leaf=min_samples_leaf,
        random_state=42,
    )
    model.fit(X_train_live, y_train_live)

    y_pred_val = model.predict(X_val_live)

    df_val_live = (
        pd.DataFrame(
            {
                "date":   dates_val_live,
                "symbol": tickers_val_live,
                "y_true": y_val_live,
                "y_pred": y_pred_val,
            }
        )
        .set_index(["date", "symbol"])
        .sort_index()
    )

    # build long/short portfolios on validation (net of transaction costs)
    _, long_val, long_short_val = compute_cs_daily_returns(
        df_val_live,
        q=q,
        horizon=lookahead,
        cost_bps=COST_BPS,
    )

    ret_series = long_short_val.replace([np.inf, -np.inf], np.nan).dropna()
    if len(ret_series) < 20:
        return 0.0  # treat as bad if too little data

    return -sharpe_ratio_np(ret_series.values)  # Optuna MINIMIZES

study_live = optuna.create_study(direction="minimize")
study_live.optimize(objective_tree_live, n_trials=20)

print("Best live params:", study_live.best_params)
print("Best live val -Sharpe:", study_live.best_value)

best_live = study_live.best_params.copy()
q_live = best_live.pop("q")

# 3) Train final tree on ALL history before as_of_date
tree_live = HistGradientBoostingRegressor(
    **best_live,
    random_state=42,
)

X_hist_live = X[hist_mask]
y_hist_live = y[hist_mask]
tree_live.fit(X_hist_live, y_hist_live)

# 4) Predictions for the as-of date (our "today")
live_mask = dates == as_of_date
X_live = X[live_mask]
tickers_live = tickers[live_mask]

y_pred_live = tree_live.predict(X_live)

panel_live = pd.DataFrame(
    {
        "ticker": tickers_live,
        "pred_fwd_21": y_pred_live,   # model prediction: 21d total return (gross)
    }
).set_index("ticker")

# approximate net-of-fee 21d return for a long position
panel_live["pred_fwd_21_net_long"] = (1.0 + panel_live["pred_fwd_21"]) * (1.0 - COST_BPS) - 1.0
panel_live["pred_daily_net_long"] = (1.0 + panel_live["pred_fwd_21_net_long"]) ** (1.0 / lookahead) - 1.0

# cross-sectional average daily net return (like EW benchmark)
mean_daily_net = panel_live["pred_daily_net_long"].mean()
panel_live["edge_vs_eqw_daily"] = panel_live["pred_daily_net_long"] - mean_daily_net

# 5) Sort and show top/bottom N
panel_sorted = panel_live.sort_values("pred_fwd_21_net_long", ascending=False)

top_long  = panel_sorted.head(TOP_N_LONG).copy()
bottom_short = panel_sorted.tail(TOP_N_SHORT).copy()

print("\n=== Paper-trade recommendation ===")
print(f"As-of date: {as_of_date.date()}  |  lookahead: {lookahead} days")
print(
    f"Tuned q_live = {q_live:.3f}  "
    f"(strategy would typically long/short ~{int(q_live * len(panel_sorted))} names per side)"
)
print(f"Assumed round-trip cost: {COST_BPS * 1e4:.1f} bps per 21d holding period")

print("\nTop tickers to LONG (preview, net of costs):")
display(
    top_long[
        ["pred_fwd_21", "pred_fwd_21_net_long", "pred_daily_net_long", "edge_vs_eqw_daily"]
    ].sort_values("pred_fwd_21_net_long", ascending=False)
)

print("\nBottom tickers to SHORT (preview, net of costs):")
# for shorts, a strongly negative predicted return is good
display(
    bottom_short[
        ["pred_fwd_21", "pred_fwd_21_net_long", "pred_daily_net_long", "edge_vs_eqw_daily"]
    ].sort_values("pred_fwd_21_net_long")  # most negative first
)


In [ ]:
# =============================================================================
# Final live model: global tuning on full history, then train once & save
# =============================================================================

# 1) Global train/validation split over all dates (for hyperparameter tuning)
all_dates = np.array(sorted(dates.unique()))
split_idx = int(len(all_dates) * 0.7)
train_end_global = all_dates[split_idx]

mask_train_global = dates <= train_end_global
mask_val_global   = dates > train_end_global

X_train_g, y_train_g = X[mask_train_global], y[mask_train_global]
X_val_g,   y_val_g   = X[mask_val_global],   y[mask_val_global]

dates_val_g   = dates[mask_val_global]
tickers_val_g = tickers[mask_val_global]

print("Global tuning:")
print("  Train dates:", dates[mask_train_global].min(), "->", dates[mask_train_global].max())
print("  Val   dates:", dates_val_g.min(), "->", dates_val_g.max())
print("  Train samples:", len(y_train_g), " Val samples:", len(y_val_g))


def objective_tree_global(trial):
    # --- Tree hyperparameters ---
    max_depth = trial.suggest_int("max_depth", 2, 8)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.2, log=True)
    max_iter = trial.suggest_int("max_iter", 100, 500)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 20, 200)
    # long/short bucket size
    q = trial.suggest_float("q", 0.05, 0.3)

    model = HistGradientBoostingRegressor(
        max_depth=max_depth,
        learning_rate=learning_rate,
        max_iter=max_iter,
        min_samples_leaf=min_samples_leaf,
        random_state=42,
    )
    model.fit(X_train_g, y_train_g)

    # Predictions on validation
    y_pred_val = model.predict(X_val_g)

    df_val_g = (
        pd.DataFrame(
            {
                "date":   dates_val_g,
                "symbol": tickers_val_g,
                "y_true": y_val_g,
                "y_pred": y_pred_val,
            }
        )
        .set_index(["date", "symbol"])
        .sort_index()
    )

    # Build long-only & long-short portfolios on validation, NET of costs
    _, long_val_g, long_short_val_g = compute_cs_daily_returns(
        df_val_g,
        q=q,
        horizon=lookahead,
        cost_bps=COST_BPS,
    )

    # We optimize **long-only** net Sharpe (you can switch to long_short_val_g if you want)
    ret_series = long_val_g.replace([np.inf, -np.inf], np.nan).dropna()
    if len(ret_series) < 20:
        return 0.0

    return -sharpe_ratio_np(ret_series.values)


study_global = optuna.create_study(direction="minimize")
study_global.optimize(objective_tree_global, n_trials=20)

print("Global best params:", study_global.best_params)
print("Global best -Sharpe (long-only, net):", study_global.best_value)

best_params_live = study_global.best_params.copy()
q_live = best_params_live.pop("q")


# 2) Train final model on ALL labeled history (X, y)
tree_cs_final = HistGradientBoostingRegressor(
    **best_params_live,
    random_state=42,
)
tree_cs_final.fit(X, y)

print("Final model trained on:", dates.min(), "->", dates.max())
print("Final model train R^2:", tree_cs_final.score(X, y))


# 3) Save bundle to disk for inference
models_dir = Path(PROJECT_ROOT) / "models"
models_dir.mkdir(exist_ok=True)

bundle = {
    "model": tree_cs_final,
    "q_live": q_live,
    "lookahead": lookahead,
    "cost_bps": COST_BPS,
    "features": CROSS_FEATURES,
    "train_start": str(dates.min().date()),
    "train_end":   str(dates.max().date()),
    "optuna_best_params": study_global.best_params,
    "optuna_best_value":  float(study_global.best_value),
}

out_path = models_dir / "sp500_tree_cs_21d_live.pkl"
joblib.dump(bundle, out_path)

print("Saved live model bundle to:", out_path)
print("q_live (fraction of names long/short):", q_live)
